In [ ]:

drop table bossbunch_db.customization ;
create TABLE bossbunch_db.customization
(
  `pers_id` varchar(10) NOT NULL,
  `image` varchar(100) DEFAULT NULL,
  `bottle_name` varchar(1000) DEFAULT NULL,
  `price` double null,
   PRIMARY KEY (`pers_id`)
)
;

INSERT INTO bossbunch_db.customization
SELECT 'PERS001','cust_bottle1.png','Wild Birthday!',24.99
union
SELECT 'PERS002','cust_bottle2.png','Symbol of remembrance!',20.45
union
 SELECT 'PERS003','cust_bottle3.png','Iconic Black!',29.99
 union
 SELECT 'PERS004','cust_bottle4.png','Celebrate together!',19.99
 union
 SELECT 'PERS005','cust_bottle5.png','Graduation YAYY!',23
 ;
 
drop table bossbunch_db.`Cart` ;
CREATE TABLE bossbunch_db.`Cart` (
  `CartID` int NOT NULL AUTO_INCREMENT,
  `CustomerID` int NOT NULL,
  `WineID` varchar(25) NOT NULL,
  `Quantity` int NOT NULL,
  `total_wine_price` float,
  `pers_id` varchar(25) ,
  `custom_message` varchar(1000),
  `gift_wrap` boolean,
  `customization_price` float,
  PRIMARY KEY (`CartID`),
  KEY `fk_cart_customer` (`CustomerID`),
  KEY `fk_cart_wine` (`WineID`),
  CONSTRAINT `fk_cart_customer` FOREIGN KEY (`CustomerID`) REFERENCES `Customer` (`CustomerID`),
  CONSTRAINT `fk_cart_wine` FOREIGN KEY (`WineID`) REFERENCES `Wine` (`WineID`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci
;

insert into bossbunch_db.`Cart`
select 1 AS CartID, 1001 AS CustomerID, "WINE001" AS WineID,1 AS Quantity,208.99 AS total_wine_price,'' ,'',FALSE,0
;

SELECT * FROM  bossbunch_db.`Cart`;


drop table bossbunch_db.Orders;
CREATE TABLE bossbunch_db.`Orders` (
  `OrderID` varchar(25) NOT NULL,
  `Date` date NOT NULL,
  `Instructions` varchar(30) DEFAULT NULL,
  `TotalAmount` float NOT NULL,
  `Coupon` varchar(30) DEFAULT NULL,
  `CustomerID` int NOT NULL,
  PRIMARY KEY (`OrderID`),
  KEY `OrdersFK1` (`CustomerID`),
  CONSTRAINT `OrdersFK1` FOREIGN KEY (`CustomerID`) REFERENCES `Customer` (`CustomerID`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci
;


CREATE TABLE bossbunch_db.Custom_orders (
  `Custom_OrderID` varchar(25) NOT NULL,
  `Date` date NOT NULL,
  `pers_id` varchar(30) DEFAULT NULL,
 `custom_message` varchar(1000),
    `gift_wrap` Boolean,
    gift_wrap_cost float,
  `custom_bottle_price` float,
    `total_custom_price` float ,
  `OrderID` varchar(25) NOT NULL,
  PRIMARY KEY (`Custom_OrderID`),
  KEY `OrdersFK` (`OrderID`),
  CONSTRAINT `OrdersFK` FOREIGN KEY (`OrderID`) REFERENCES `Orders` (`OrderID`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci
;

DROP TABLE bossbunch_db.Sells;
DROP TABLE bossbunch_db.Items;
CREATE TABLE bossbunch_db.Items (
  `ItemID` varchar(25),
  `quantity` int NOT NULL,
  `OrderID` varchar(25) NOT NULL,
  -- PRIMARY KEY (`ItemID`),
  KEY `OrdersFKI` (`OrderID`),
  CONSTRAINT `OrdersFKI` FOREIGN KEY (`OrderID`) REFERENCES `Orders` (`OrderID`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci
;


DROP TABLE bossbunch_db.Orders_CreditCard;
CREATE TABLE bossbunch_db.Orders_CreditCard (
  `CreditCard` varchar(25) NOT NULL,
  `OrderID` varchar(25) NOT NULL,
    `CustomerID`  int NOT NULL,
  PRIMARY KEY (`CreditCard`),
  KEY `OrdersFKC` (`OrderID`),
    KEY `fk_credit_cart_customer` (`CustomerID`),
    CONSTRAINT `fk_credit_cart_customer` FOREIGN KEY (`CustomerID`) REFERENCES `Customer` (`CustomerID`),
  CONSTRAINT `OrdersFKC` FOREIGN KEY (`OrderID`) REFERENCES `Orders` (`OrderID`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_0900_ai_ci
;

PROCEDURE:
    
drop procedure get_cart_details;
    
DELIMITER //

CREATE PROCEDURE get_cart_details(IN cust_id INT)
BEGIN
    select CustomerID
    ,w.WineName
    , sum(quantity)
    ,sum(w.price) as total_wine_price
    ,case when bottle_name is not null then bottle_name else "" end as bottle_name
    ,case when sum(c.customization_price) is not null then sum(c.customization_price) else 0.00 end as customization_price
    ,sum(case when gift_wrap is True then 3.99 else 0 end ) as gift_wrap_cost
    ,case when custom_message is not null then custom_message else "" end as custom_message
    from bossbunch_db.Cart c
	inner join bossbunch_db.Wine w
	on c.WineID = w.WineID
    left join bossbunch_db.customization pers
    on c.pers_id= pers.pers_id
	where CustomerID=cust_id
	group by CustomerID,w.WineName,bottle_name,custom_message;
END //

DELIMITER ;


